<a href="https://colab.research.google.com/github/vedrocks15/Qiskit_Circuits/blob/master/Quanvolutional_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Try to see the variations in the performace gains offered by a random quantum circuit to extract features

In [ ]:
! pip install pennylane

In [ ]:
# qml used to register qubit as devices
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers # helps in the generation of a random circuit

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

HyperParameters

In [ ]:
n_epochs = 30    # Number of optimization epochs
lr       = 0.05  # Learning rate
n_layers = 1     # Number of random layers
n_train  = 50
n_test   = 10

Loading & pre-processing MNIST Data

In [ ]:
mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255


train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Add extra dimension for convolution channels
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

In [ ]:
print(len(train_images))
print(len(test_images))

50
10


Building of Random Quantum Circuit

In [ ]:
# The quanvolutional filter would be of the size 2x2 therefore we make use of 4 wires
dev = qml.device("default.qubit", wires=4)

# The qnode generted will have the following gate properties :
# 1) Encoding  : Ry rotation gate where the angles are scaled by PI
# 2) Ciruit    : Of n_layers random in nature
# 3) Decoding  : 4 expectation value measurements

rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, 4))
# decorator for quantum node
@qml.qnode(dev)
def circuit(phi=None):
    # Encoding
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit generation
    RandomLayers(rand_params, wires=list(range(4)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]



In [ ]:
def quan_filter(image):

    # 2x2 filter with stride 1 and passes the data through the quantum circuit
    # the output of the qunatum ciruit is 4 that is mapped into 4 different channels indicating the same pixel
    out = np.zeros((14, 14, 4))

    # steps of 2
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
          # The cicuit remains the same because the uniform random sampling is done only once
            q_results = circuit(phi=[image[j, k, 0], image[j, k + 1, 0], image[j + 1, k, 0], image[j + 1, k + 1, 0]])
            # appropriate expectation value is assigned to right channel
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

The experiment does not modify the paramters of the quantum circuit

In [ ]:
q_train_images = list()
print("Quantum pre-processing of train images:")
for idx, img in enumerate(train_images):
  print("{}/{}        ".format(idx + 1, n_train))
  q_train_images.append(quan_filter(img))
q_train_images = np.asarray(q_train_images)

q_test_images = list()
print("\nQuantum pre-processing of test images:")
for idx, img in enumerate(test_images):
  print("{}/{}        ".format(idx + 1, n_test))
  q_test_images.append(quan_filter(img))
q_test_images = np.asarray(q_test_images)


Quantum pre-processing of train images:
1/50        
2/50        
3/50        
4/50        
5/50        
6/50        
7/50        
8/50        
9/50        
10/50        
11/50        
12/50        
13/50        
14/50        
15/50        
16/50        
17/50        
18/50        
19/50        
20/50        
21/50        
22/50        
23/50        
24/50        
25/50        
26/50        
27/50        
28/50        
29/50        
30/50        
31/50        
32/50        
33/50        
34/50        
35/50        
36/50        
37/50        
38/50        
39/50        
40/50        
41/50        
42/50        
43/50        
44/50        
45/50        
46/50        
47/50        
48/50        
49/50        
50/50        

Quantum pre-processing of test images:
1/10        
2/10        
3/10        
4/10        
5/10        
6/10        
7/10        
8/10        
9/10        
10/10        


Classical Model 

In [ ]:
model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])

model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

In [ ]:
model.fit(q_train_images,
    train_labels,
    validation_data=(q_test_images, test_labels),
    batch_size=4,
    epochs=n_epochs,
    verbose=2,
    )

Epoch 1/30
13/13 - 0s - loss: 8.2076 - accuracy: 0.0600 - val_loss: 8.5833 - val_accuracy: 0.3000
Epoch 2/30
13/13 - 0s - loss: 3.8927 - accuracy: 0.3400 - val_loss: 8.2895 - val_accuracy: 0.1000
Epoch 3/30
13/13 - 0s - loss: 3.6108 - accuracy: 0.3800 - val_loss: 3.4312 - val_accuracy: 0.5000
Epoch 4/30
13/13 - 0s - loss: 0.3983 - accuracy: 0.9000 - val_loss: 0.8217 - val_accuracy: 0.7000
Epoch 5/30
13/13 - 0s - loss: 0.1254 - accuracy: 0.9600 - val_loss: 1.6893 - val_accuracy: 0.4000
Epoch 6/30
13/13 - 0s - loss: 0.0598 - accuracy: 1.0000 - val_loss: 0.8443 - val_accuracy: 0.8000
Epoch 7/30
13/13 - 0s - loss: 0.0223 - accuracy: 1.0000 - val_loss: 0.7490 - val_accuracy: 0.8000
Epoch 8/30
13/13 - 0s - loss: 0.0184 - accuracy: 1.0000 - val_loss: 0.7137 - val_accuracy: 0.8000
Epoch 9/30
13/13 - 0s - loss: 0.0180 - accuracy: 1.0000 - val_loss: 0.7414 - val_accuracy: 0.8000
Epoch 10/30
13/13 - 0s - loss: 0.0160 - accuracy: 1.0000 - val_loss: 0.7242 - val_accuracy: 0.8000
Epoch 11/30
13/13 -